# Homework 2: Word Similarity

In [ ]:
import warnings
warnings.filterwarnings('ignore')


In [ ]:
cd ./Downloads/

Student Name: Udit Goel

Student ID: 1042890

## General info

<b>Due date</b>: Thursday, 4 June 2020 5pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -10% per day (both week and weekend days counted)

<b>Marks</b>: 10% of mark for class (with 9% on correctness + 1% on quality and efficiency of your code)

**Note**: As we will be implementing neural networks in this assignment, you're encouraged to build your notebook on **colab**. See the programming exercise in workshop-07 (`10-bert.ipynb`) if you are not familiar with colab.

<b>Materials</b>: See [Using Jupyter Notebook and Python page](https://canvas.lms.unimelb.edu.au/courses/17601/pages/using-jupyter-notebook-and-python?module_item_id=1678430) on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. Deep learning libraries such as keras and pytorch are also allowed.  You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.




<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

## Overview

In this homework, you'll be quantifying the similarity between pairs of words of a dataset using different methods with the word co-occurrence in the Brown corpus and synset structure of WordNet. Firstly, you will preprocess the dataset to filter out the rare and ambiguous words. Secondly, you will calculate the similarity scores for pairs of words in the filtered dataset using Lin similarity, NPMI and LSA. Lastly, you will quantify how well these methods work by comparing to a human annotated gold-standard.

## 1. Preprocessing (2 marks)

## Question 1 (1.0 mark)

<b>Instructions</b>: For this homework we will be comparing our methods against a popular dataset of word similarities called <a href="http://alfonseca.org/eng/research/wordsim353.html">Similarity-353</a>. You need to first obtain this dataset, which is on Canvas (assignment 2). The file we will be using is called *set1.tab*. Make sure you save this in the same folder as the notebook.  Except for the header (which should be stripped out), the file is tab formated with the first two columns corresponding to two words, and the third column representing a human-annotated similarity between the two words. <b>You should ignore the subsequent columns</b>.

Here shows the first six lines of the file:

```
Word 1	Word 2	Human (mean)	1	2	3	4	5	6	7	8	9	10	11	12	13	
love	sex	6.77	9	6	8	8	7	8	8	4	7	2	6	7	8	
tiger	cat	7.35	9	7	8	7	8	9	8.5	5	6	9	7	5	7	
tiger	tiger	10.00	10	10	10	10	10	10	10	10	10	10	10	10	10	
book	paper	7.46	8	8	7	7	8	9	7	6	7	8	9	4	9	
computer	keyboard	7.62	8	7	9	9	8	8	7	7	6	8	10	3	9	
```
    
You should load this file into a Python dictionary (NOTE: in Python, tuples of strings, i.e. ("tiger","cat") can serve as the keys of a dictionary to map to their human-annotated similarity). This dataset contains many rare words: we need to filter this dataset in order for it to be better suited to the resources we will use in this assignment. So your first goal is to filter this dataset to generate a smaller test set where you will evaluate your word similarity methods.

The first filtering is based on **document frequencies** (not token frequencies) in the Brown corpus, in order to remove rare words. In this homework, **we will be treating the paragraphs of the Brown corpus as our "documents"**. You can iterate over them by using the `paras` method of the corpus reader. You should remove tokens that are not alphabetic. Tokens should be lower-cased and lemmatized (lemmatize function provided). Store this preprocessed data in *brown_corpus* object (it will be used for question 4 and 5 later).

Now calculate document frequencies for each word type, and use this to remove from your word similarity data any word pairs where at least one of the two words has a document frequency of **$< 8$** in this corpus. You should store all the word pair and similarity mappings in your filtered test set in a dictionary called *filtered_gold_standard*.

Note: the document frequency of a word denotes the number of documents that contains the word.

**Task**: Filter word pairs from *set1.tab* based on document frequencies. Produce *brown_corpus*, which is a list where each element is a set of words for one paragraph (e.g. the first element in *brown_corpus* should contain all the unique word types for the first paragraph). Produce *filtered_gold_standard*, a dictionary of filtered word pairs with human similarity ratings (the dictionary should have (word1, word2) as keys, and similarity ratings as values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *filtered_gold_standard*.

In [ ]:
import nltk
import csv
from nltk.corpus import brown
from nltk.corpus import wordnet
from collections import Counter
nltk.download("brown")
nltk.download("wordnet")

# filtered_gold_standard stores the word pairs and their human-annotated similarity in your filtered test set
filtered_gold_standard = {}

# lemmatizer
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatize(word):
    
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma


# Your answer BEGINS HERE
set1_df = {}

##Reading set1 file
with open('set1.tab', newline='') as file:
    file = csv.reader(file, delimiter='\t')
    next(file) #Missing the first row
    for row in file:
        set1_df[(row[0],row[1])] = float(row[2])



def preprocessmap(word):
    if word.isalpha(): #Checking if word is alphabetic or not
        word = word.lower() #Converting to lower case
        return lemmatize(word)


def flatten_lists(lists): #function to flatten collection of lists
    final_list=[]
    for i in range(len(lists)):
        final_list.extend(lists[i])
    return final_list

def mylamda(row): #Function to ceck if the condition is meeting or not. 
    d1,d2=0,0
    if row[0] in document_frequency:
        d1 = document_frequency[row[0]] 
    if row[1] in document_frequency :
        d2 = document_frequency[row[1]]
    return (d1 >=  8 and d2 >= 8)

document_frequency = {} #Initialising empty dictionary

brown_cor = [list(map(preprocessmap,flatten_lists(i))) for i in brown.paras()] #Applying preprocesssing to flattened lists

#Removing elements which are not following condition
brown_corpus = [list(set(filter(lambda x: x is not None, i))) for i in brown_cor]

for i in brown_corpus: #Loop to make document frequency
    for j in i:
        if j not in document_frequency:
            document_frequency[j] = 1
        else:
            document_frequency[j] += 1

filtered_gold_standard = {k:v for (k,v) in set1_df.items() if mylamda(k) == True }

# Your answer ENDS HERE

print(len(filtered_gold_standard))
print(filtered_gold_standard)



<b>For your testing: </b>

In [ ]:
assert(len(brown_corpus)==15667)
assert(len(filtered_gold_standard) > 50 and len(filtered_gold_standard) < 100)
assert(filtered_gold_standard[('love', 'sex')] == 6.77)

### Question 2 (1.0 mark)

<b>Instructions</b>: Here, you apply the second filtering. The second filtering is based on words with highly ambiguous senses and involves using the NLTK interface to WordNet. Here, you should remove any words which do not have a **single primary sense**. We define single primary sense here as either: (a) having only one sense (i.e. only one synset), or (b) where the count (as provided by the WordNet `count()` method for the lemmas associated with a synset) of the most common sense is at least 4 times larger than the next most common sense. Note that a synset can be associated with multiple lemmas. You should only consider the count of your lemma.

Note: You should lowercase the lemmas of a synset when matching your word; and if there are multiple lowercased lemmas that match your word, you should sum up the count of all matching lemmas.

Additionally, you should remove any words where the primary sense is **not a noun** (this information is also in the synset). Store the synset corresponding to this primary sense in a dictionary for use in the next section.

Given this definition, remove the word pairs from the test set where at least one of the words does not meet the above criteria. When you have applied the two filtering steps, you should store all the word pair and similarity mappings in your filtered test set in a dictionary called *final_gold_standard*.

**Task**: Filter word pairs for any words which do not have a single primary sense and aren't nouns. Produce *final_gold_standard*, a dictionary of filtered word pairs with human similarity ratings. Note: this second filtering is applied on top of the first filtering (question 1). In other words, you shouldn't consider any word pairs that have already been discarded by the first filtering in this question.

**Check**: Use the assertion statements in *"For your testing"* for the expected *final_gold_standard*.

In [ ]:
# final_gold_standard stores the word pairs and their human-annotated similarity in your final filtered test set
final_gold_standard = {}
word_primarysense = {} #a dictionary of (word, primary_sense) (used for next section); primary_sense is a synset
 

# Your answer BEGINS HERE
def count_lemmas(syn, word):
    total = 0
    for i in syn.lemmas():
        if i.name().lower() == word.lower():
            return i.count()

def find_count_lemma(synset, word):
    count = 0
    for i in synset.lemmas():
        if i.name().lower() == word.lower():
            return count
    
def two_most_common_words(word):
    most_common = None
    second_common = None 
    d1 = sorted(wordnet.synsets(word),key=lambda x: count_lemmas(x, word),reverse=True)
    most_common = d1[0]
    for i in d1 :
        
        if i.name().lower() != most_common.name().lower():
            second_common = i
            break
    condition1 = (len(d1) == 1)
    if most_common is not None and second_common is not None:
        condition2 = (count_lemmas(most_common,word) >= 4*count_lemmas(second_common, word))
    condition3 = (most_common.pos() == 'n')
    if (condition1 or condition2) and condition3:
        word_primarysense[word] = most_common
    return (condition1 or condition2) and condition3
    


final_gold_standard = {k:v for (k,v) in filtered_gold_standard.items() if (two_most_common_words(k[0]) and \
                   two_most_common_words(k[1]))}


# Your answer ENDS HERE
print(len(final_gold_standard))
print(final_gold_standard)






<b>For your testing:</b>

In [ ]:
assert(len(final_gold_standard) > 10 and len(final_gold_standard) < 40)
assert(final_gold_standard[('professor', 'doctor')] == 6.62)

## 2. Computing word similiarity with Lin similarity, NPMI and LSA (3 marks)

### Question 3 (1.0 mark)

<b>Instructions</b>: Now you will create several dictionaries with similarity scores for pairs of words in your test set derived using the techniques discussed in class. The first of these is the Lin similarity for your word pairs using the information content of the Brown corpus, which you should calculate using the primary sense for each word derived above. You can use the built-in method included in the NLTK interface, you don't have to implement your own. 

When you're done, you should store the word pair and similarity mappings in a dictionary called *lin_similarities*.

**Task**: Compute word pair similarity using Lin similarity for the test set. Produce *lin_similarities*, a dictionary of word pairs (keys) and computed Lin similarity scores (values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *lin_similarities*. 

In [ ]:
from nltk.corpus import wordnet_ic
nltk.download('wordnet_ic')
semcor_ic = wordnet_ic.ic('ic-brown.dat')
lin_similarities = {}


# Your answer BEGINS HERE

for i in final_gold_standard.keys():
    synset1 = word_primarysense[i[0]]
    synset2 = word_primarysense[i[1]]
    similarity = synset1.lin_similarity(synset2, semcor_ic)
    lin_similarities[(i[0],i[1])] = similarity

# Your answer ENDS HERE


print(lin_similarities)

<b>For your testing:</b>

In [ ]:
assert(lin_similarities[('professor', 'doctor')] > 0.5 and lin_similarities[('professor', 'doctor')] < 1)

### Question 4 (1.0 mark)

**Instructions:** Next, you will calculate Normalized PMI (NPMI) for your word pairs using word frequency derived from the Brown.

PMI is defined as:

\begin{equation*}
PMI = \log_2\left(\frac{p(x,y)}{p(x)p(y)}\right)
\end{equation*}

where

\begin{equation*}
p(x,y) = \frac{\text{Number of paragraphs with the co-occurrence of x and y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(x) = \frac{\text{Number of paragraphs with the occurrence of x}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(y) = \frac{\text{Number of paragraphs with the occurrence of y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

with the sum over $i$ ranging over all paragraphs. Note that there are other ways PMI could be formulated.

NPMI is defined as:

\begin{equation*}
NPMI = \frac{PMI}{-log_2(p(x,y))} = \frac{log_2(p(x)p(y))}{log_2(p(x,y))} - 1
\end{equation*}

Thus, when there is no co-occurrence, NPMI is -1. NPMI is normalized between [-1, +1]. 

You should use the *brown_corpus* object you've created in question 1 as your corpus here. You need to use the basic method for calculating PMI introduced in class (and also in the reading) which is appropriate for any possible definition of co-occurrence (here, there is co-occurrence when a word pair appears in the same paragraph), but you should only calculate PMI for the words in your test set. You must avoid building the entire co-occurrence matrix, instead you should keeping track of the sums you need for the probabilities as you go along. 

When you have calculated NPMI for all the pairs, you should store the word pair and NPMI-similarity mappings in a dictionary called *NPMI_similarities*.

**Task**: Compute word pair similarity using NPMI similarity for the test set. Produce *NPMI_similarities*, a dictionary of word pairs (keys) and computed NPMI similarity scores (values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *NPMI_similarities*.

In [ ]:
import math
# NPMI_similarities stores the word pair and NPMI similarity mappings
NPMI_similarities = {}

total_words = 0
for i in brown_corpus:
    total_words += len(i)

def cocurrence(word1, word2):
    total_cocurrence = 0
    word1_occurence = 0
    word2_occurence = 0
    for i in brown_corpus:
        if word1.lower() in i :
            word1_occurence += 1 #Calculating word1 frequency
        if word2.lower() in i:
            word2_occurence += 1 #Calculating word2 frequency
        if word1.lower() in i and word2.lower() in i:
            total_cocurrence += 1 #Calculating co-occurence frequency
    co_prob =  total_cocurrence/total_words
    prob_word1 = word1_occurence/total_words
    prob_word2 = word2_occurence/total_words
    if total_cocurrence == 0: #If no co-occurence then return -1 
        return -1
    else:
        pmi = math.log((co_prob/(prob_word1*prob_word2)),2)
        npmi = pmi / (- math.log(co_prob,2))
        return npmi

for i in final_gold_standard.keys():
    NPMI_similarities[(i[0],i[1])] = cocurrence(i[0],i[1])
# Your answer BEGINS HERE




# Your answer ENDS HERE


print(NPMI_similarities)

<b>For your testing:</b>

In [ ]:
assert(NPMI_similarities[('professor', 'doctor')] == -1)

### Question 5 (1.0 mark)

**Instructions:** Here we'll use singular value decomposition (SVD) to derive similarity scores using the Latent Semantic Analysis (LSA) method. Recall that LSA applies SVD and truncation to get a dense vector representation of a word type. To measure similarity between two words we calculate cosine similarity between the LSA vectors of the words.

We'll first build a term-document frequency matrix (as before, a document is a paragraph in Brown corpus). That is, the rows corresponds to words in the vocabulary, and the columns represent the paragraphs/documents in Brown corpus. Each cell records the document frequency of a word (1 if the word appears in the document, 0 otherwise). You should use the *brown_corpus* object created in question 1 as your corpus to build the term-document frequency matrix.

Given the term-document frequency matrix, we'll use `truncatedSVD` in `sklearn` to produce dense vectors of length k = 500, and then use cosine similarity to produce similarities for the word pairs in the test set.

You can use the workshop's notebook on distributional similarity (``09-distributional-similarity.ipynb``) as a starting point, but note that we use term-document frequency matrix (as opposed to the tf-idf matrix in the workshop).

When you are done, you should store the word pair and LSA-similarity mappings in a dictionary called *LSA_similarities*.

**Task**: Compute word pair similarity using LSA. Produce *LSA_similarities*, a dictionary of word pairs (keys) and computed LSA similarity scores (values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *LSA_similarities*. 



In [ ]:
import numpy as np
from numpy.linalg import norm
from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
# LSA_similarities stores the word pair and LSA similarity mappings
LSA_similarities = {}

# Your answer BEGINS HERE

def cos_sim(a, b): #function to calculate cosine similarity
    return np.dot(a, b)/(norm(a)*norm(b))

def get_BOW(text): #This functions gives bag of words
    BOW = {}
    for word in text:
        BOW[word.lower()] =  1
    return BOW

texts = []
for fileid in brown_corpus:
    texts.append(get_BOW(fileid))

vectorizer = DictVectorizer(sparse=False)

brown_matrix = vectorizer.fit_transform(texts)
brown_matrix_transposed = csr_matrix(brown_matrix).transpose()
svd = TruncatedSVD(n_components=500)
brown_matrix_lowrank = svd.fit_transform(brown_matrix_transposed)
    
for i in final_gold_standard.keys():
    a = brown_matrix_lowrank[vectorizer.vocabulary_[i[0]]]
    b = brown_matrix_lowrank[vectorizer.vocabulary_[i[1]]]
    LSA_similarities[(i[0],i[1])] = cos_sim(a,b)

# Your answer ENDS HERE


print(LSA_similarities)

<b>For your testing:</b>

In [ ]:
assert(LSA_similarities[('professor', 'doctor')] > 0 and LSA_similarities[('professor', 'doctor')] < 0.4)

## Computing word similarity with feedforward language model (3 marks)

### Question 6 (1.0 mark)

**Instructions**: Here we'll build a n-gram neural language model to learn word embeddings, and compute word similarity based on the word embeddings. As before we will use the Brown corpus as training data for our language model, and the first step is to collect the vocabulary.

As before, we'll treat paragraphs in the Brown corpus as our documents. The first 12K paragraphs/documents will serve as our training data, and the rest (3K+ documents) as development data. The first step towards building a language model is to collect the vocabulary, i.e. the set of unique word types in our training data. When collecting the word types, you should lowercase all words, and only keep word types that have a frequency $>= 5$. Store your vocabulary in the _vocab_ object.

Note: 
  - we'll be using _$<$UNK$>$_ to represent unseen words, and so _vocab_ is initialised with the special _$<$UNK$>$_.
  - you do not need to do any other additional preprocessing aside from the aforementioned steps (e.g. no need to remove symbols, etc)
  - you should not use *brown_corpus* here, as you need the words in their original order (*brown_corpus* stores a set of words for each paragraph, and so do not contain word order information)

**Task**: Collect a set of unique word types in the training portion (first 12K paragraphs) of the Brown corpus. Produce _vocab_, which is a set that contains all the word types.

**Check**: Use the assertion statements in *"For your testing"* below to check the vocabulary size.

In [ ]:
num_train = 12000
UNK_symbol = "<UNK>"
vocab = set([UNK_symbol])


# Your answer BEGINS HERE
training_data = brown.paras()[:num_train]  #Splitting training data
development_set = brown.paras()[num_train:] #Spliting development data
term_frequency  = {}
for i in training_data: 
    lists  = flatten_lists(i)
    for j in lists:
        j = j.lower()
        if j not in term_frequency: #Building term frequency data
            term_frequency[j] = 1
        else:
            term_frequency[j] += 1
for key,value in term_frequency.items():
    if value >= 5:
        vocab.add(key.lower())
        

# Your answer ENDS HERE


print(len(vocab))

**For your testing:**

In [ ]:
assert(len(vocab) > 8000 and len(vocab) < 20000)

### Question 7 (1.0 mark)

**Instructions**: As we'll be building a trigram neural language model (based on lecture 7, page 20), the next step is to collect trigrams to construct our training data. In a trigram neural language model, for example if we have the trigram _cow eats grass_, the input to the model is the first two terms of a trigram (_cow_ and _eats_), and the language model's aim is to predict the last term of the trigram (_grass_). Your task here is to construct the training and development data for the language model. Just like the previous step, the first 12K paragraphs will serve as our training data, and the remaining 3K+ will be for development. You'll need to map words into IDs when constructing the training and development data. Any words that are not in _vocab_ should be mapped to the special _$<$UNK$>$_ symbol.

As an example, given the sentence "_a big fat hungry cow ._", you should create the following training examples:

|input|target|
|:---:|:----:|
|<i>a</i>, _big_|_fat_|
|_big_, _fat_|_hungry_|
|_fat_, _hungry_|_cow_|
|_hungry_, _cow_|_._|


Note:
   - _vocab_ is a set and so does not map words into IDs. You'll need to create a word to ID mapping first based on _vocab_.
   - A trigram should not cross sentence boundary.
   - You should ignore sentences that have less than 3 words (as they are too short to form trigrams).
   - We won't need special starting symbol when collecting the trigrams, as we are only interested in learning word embeddings here (and not computing probabilities of a sentence).


**Task**: Create training and development data. The training input and target data should be stored in the <i>x_train</i> and <i>y_train</i> respectively, and development input and target data in <i>x_dev</i> and <i>y_dev</i> respectively.

**Check**: Use the assertion statements in *"For your testing"* below for the expected shape of the outputs.

In [ ]:
from nltk.util import ngrams
# Your answer BEGINS HERE
indexes = list(range(0, len(vocab)))
vocab_dict =  dict(zip(vocab, indexes))
x_train = []
y_train = []
for k,i in enumerate(training_data): #Makes training data
    lists = flatten_lists(i)
    for i in ngrams(lists,3):
        
        if i[0].lower() not in vocab_dict:
            train_1 = vocab_dict['<UNK>']
        else:
            train_1 = vocab_dict[i[0].lower()]
        if i[1].lower() not in vocab_dict:
            train_2 = vocab_dict['<UNK>']
        else:
            train_2 = vocab_dict[i[1].lower()]
        x_train.append([train_1, train_2])
        if i[2].lower() not in vocab_dict:
            test = vocab_dict['<UNK>']
        else:
            test =  vocab_dict[i[2].lower()]
        y_train.append(test)

x_dev = []
y_dev = []
for k,i in enumerate(development_set): #Makes development set
    lists = flatten_lists(i)
    
    for i in ngrams(lists,3):
        if i[0].lower() not in vocab_dict:
            train_1 = vocab_dict['<UNK>']
        else:
            train_1 = vocab_dict[i[0].lower()]
        if i[1].lower() not in vocab_dict:
            train_2 = vocab_dict['<UNK>']
        else:
            train_2 = vocab_dict[i[1].lower()]
        x_dev.append([train_1, train_2])
        if i[2].lower() not in vocab_dict:
            test = vocab_dict['<UNK>']
        else:
            test =  vocab_dict[i[2].lower()]
        y_dev.append(test)

# Your answer ENDS HERE

#Converting lists to np arrays
x_train = np.array(x_train)
y_train = np.array(y_train)
x_dev = np.array(x_dev)
y_dev = np.array(y_dev)
    
print(x_train.shape)
print(y_train.shape)
print(x_dev.shape)
print(y_dev.shape)

**For your testing:**

In [ ]:
assert(x_train.shape[0] == y_train.shape[0])
assert(x_dev.shape[0] == y_dev.shape[0])
assert(x_train.shape[0] > 500000)
assert(x_dev.shape[0] > 50000)

### Question 8 (1.0 mark)

**Instructions**: Now let's build the trigram neural language model. We'll use the language model described in lecture 7 (page 20):

$x' = e(x_1) \oplus e(x_2)$

$h = \tanh(W_1 x' + b)$

$y = $ softmax$(W_2 h)$

where $\oplus$ is the concatenation operation, $x_1$ and $x_2$ are the input words, $e$ is an embedding function, and $y$ is the target word. You can use either `keras` or `pytorch` for building your model.

Set the dimension of the word embeddings and $h$ to 100, and train your model with 3 epochs with a batch size of 256. You will not need to tune hyper-parameters for this task.

After the model is trained, use the word embeddings to compute word similarity in the test set. Store the similarity scores in a dictionary called *lm_similarities*.

Note:
  - For words in the test set that are not in your vocabulary, you should treat them as unknown words. In other words, you should use $<$UNK$>$'s embedding to represent these words.
  - The training may take some time on CPU. You can run your notebook on colab with a GPU if you want faster training (see the programming exercise in workshop-07 if you're not familiar with colab)

**Task**: Train a trigram neural language model, and use the learned word embeddings to compute word similarity. Produce *lm_similarities*, a dictionary that contains word pairs as keys and similarity scores as values.

In [ ]:
from keras.models import Sequential
from keras import layers, Input
from keras.utils import to_categorical
lm_similarities = {}
vocab_size = len(vocab_dict)

# Your answer BEGINS HERE

#Declaring a feed forward neural network model
model = Sequential()
model.add(layers.Embedding(input_dim = vocab_size, output_dim = 100, input_length = 2)) #Embedding layer
model.add(layers.Flatten()) #Flattening layer
model.add(layers.Dense(100, activation='tanh')) #Adding hidden layer
model.add(layers.Dense(vocab_size, activation = 'softmax')) #Final output layer

# Your answer ENDS HERE

model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy')
model.summary()

x_test = []
y_test = []

model.fit(x_train, y_train, epochs=3, batch_size=256, validation_data=(x_dev, y_dev))


In [25]:
embeddings = model.get_layer(index=0).get_weights()[0]
for k,i in enumerate(final_gold_standard):
    if i[0] not in vocab_dict:
        embed1 = embeddings[vocab_dict['<UNK>']]
    else:
        
        embed1 = embeddings[vocab_dict[i[0]]]
    if i[1] not in vocab_dict:
        embed2 = embeddings[vocab_dict["<UNK>"]]
    else:
        embed2 = embeddings[vocab_dict[i[1]]]
    lm_similarities[(i[0],i[1])] = cos_sim(embed1, embed2)
print(lm_similarities)

{('bread', 'butter'): 0.34102115, ('professor', 'doctor'): 0.21716338, ('student', 'professor'): 0.33623403, ('stock', 'egg'): 0.49702626, ('money', 'cash'): 0.2844231, ('king', 'queen'): 0.2530256, ('bishop', 'rabbi'): 0.25156778, ('football', 'basketball'): 0.38258144, ('football', 'tennis'): 0.35740107, ('alcohol', 'chemistry'): -0.23344013, ('baby', 'mother'): 0.06469177, ('car', 'automobile'): -0.06626839, ('journey', 'voyage'): 0.107774936, ('coast', 'shore'): -0.15673135, ('brother', 'monk'): 0.570878, ('journey', 'car'): -0.07162175, ('coast', 'hill'): 0.17835954, ('forest', 'graveyard'): 0.07754704, ('monk', 'slave'): -0.06459527, ('coast', 'forest'): 0.47430345, ('psychology', 'doctor'): 0.20336024, ('psychology', 'mind'): 0.08726869, ('psychology', 'health'): 0.40560368, ('psychology', 'science'): 0.56492317, ('planet', 'moon'): 0.4251139, ('planet', 'galaxy'): 0.15662263}


## 3. Comparison with the Gold Standard (1 mark)

### Question 9 (1.0 mark)

**Instructions:** Finally, you should compare all the similarities you've created to the gold standard you loaded and filtered in the first step. For this, you can use the Pearson correlation co-efficient (`pearsonr`), which is included in scipy (`scipy.stats`). Be careful converting your dictionaries to lists for this purpose, the data for the two datasets needs to be in the same order for correct comparison using correlation. Write a general function, then apply it to each of the similarity score dictionaries.

When you are done, you should put the result in a dictionary called *pearson_correlations* (use the keys: 'lin', 'NPMI', 'LSA', 'lm').

**Task**: Compute the Pearson correlation coefficient between the estimated similarity scores (Lin, NPMI and LSA similarities) and the gold standard similarity ratings. Produce *pearson_correlations*, a dictionary containing the methods as keys and correlations as values.

**Check**: Use the assertion statements in *"For your testing"* below for the expected *pearson_correlations*. 

<b>Hint:</b> All of the methods used here should be markedly above 0, but also far from 1 (perfect correlation); if you're not getting reasonable results, go back and check your code for bugs! 

In [ ]:
from scipy.stats import pearsonr

# pearson_correlations stores the pearson correlations with the gold standard of 'lin', 'NPMI', 'LSA', 'lm'
pearson_correlations = {}

# Your answer BEGINS HERE

def corr(dict1, dict2):
    x,y = [], []
    for i in dict1.keys():
        x.append(dict1[i])
        y.append(dict2[i])
    return pearsonr(x,y)[0]

##Adding keys to dictionary

pearson_correlations['lin'] = corr(lin_similarities, final_gold_standard)
pearson_correlations['NPMI'] = corr(NPMI_similarities, final_gold_standard)
pearson_correlations['LSA'] = corr(LSA_similarities, final_gold_standard)
pearson_correlations['lm'] = corr(lm_similarities, final_gold_standard)

# Your answer ENDS HERE


print(pearson_correlations)

<b>For your testing:</b>

In [ ]:
assert(pearson_correlations['lin'] > 0.4 and pearson_correlations['lin'] < 0.8)

## A final word

Normally, we would not use a corpus as small as the Brown for the purposes of building word vectors. Also, note that filtering our test set to just words we are likely to do well on would typically be considered cheating.